### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060


# TP3 - Problema 3:


Considere de novo o 1º problema do trabalho TP2 relativo à descrição da cifra A5/1 e o FOTS usando BitVec’s que aí foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s.
Sejam $\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial e condição de erro use os predicados
$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad e $
$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$

1. Codifique em “z3” o SFOTS assim definido.
2. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.


In [ ]:
from z3 import *

# Tamanhos dos registradores
SIZE0, SIZE1, SIZE2 = 19, 22, 23

# Posições dos bits de controle
CONTROL_BIT0, CONTROL_BIT1, CONTROL_BIT2 = 8, 10, 10

# Polinômios de retroalimentação em forma binária
POLY0 = 0b1110010000000000001      # x¹⁹ + x¹⁸ + x¹⁷ + x¹⁴ + 1
POLY1 = 0b1100000000000000000001   # x²² + x²¹ + 1
POLY2 = 0b11100000000000010000001  # x²³ + x²² + x²¹ + x⁸ + 1

# Variáveis de estado globais
LFSR0 = BitVec('lfsr0', SIZE0)
LFSR1 = BitVec('lfsr1', SIZE1)
LFSR2 = BitVec('lfsr2', SIZE2)

def get_state_vars(suffix=""):
    """
    Cria um novo conjunto de variáveis de estado.
    
    Args:
        suffix (str): Sufixo para identificar as variáveis
    
    Returns:
        dict: Dicionário com as novas variáveis de estado
    """
    return {
        'lfsr0': BitVec(f'lfsr0_{suffix}', SIZE0),
        'lfsr1': BitVec(f'lfsr1_{suffix}', SIZE1),
        'lfsr2': BitVec(f'lfsr2_{suffix}', SIZE2)
    }

def substitute_vars(formula, old_state, new_state):
    """
    Substitui variáveis em uma fórmula.
    
    Args:
        formula: Fórmula Z3 original
        old_state (dict): Estado antigo
        new_state (dict): Novo estado
    
    Returns:
        Fórmula Z3 com variáveis substituídas
    """
    substitutions = [(old_state[key], new_state[key]) for key in old_state]
    return substitute(formula, substitutions)

def trans(curr, prox):
    """
    Implementa a função de transição do cifrador de fluxo A5/1.
    
    Args:
        curr (dict): Estado atual dos registros LFSR
        prox (dict): Estado próximo dos registros
    
    Returns:
        Fórmula Z3 representando a transição
    """
    # Extrai os bits de clock de cada registro
    b0 = Extract(CONTROL_BIT0, CONTROL_BIT0, curr['lfsr0']) == 1
    b1 = Extract(CONTROL_BIT1, CONTROL_BIT1, curr['lfsr1']) == 1
    b2 = Extract(CONTROL_BIT2, CONTROL_BIT2, curr['lfsr2']) == 1

    # Implementa a regra da maioria
    majority = Or(And(b0, b1), And(b1, b2), And(b0, b2))

    # Define os polinômios de retroalimentação
    s0 = BitVecVal(POLY0, SIZE0)
    s1 = BitVecVal(POLY1, SIZE1)
    s2 = BitVecVal(POLY2, SIZE2)

    # Calcula as transições para cada LFSR
    t0 = If(majority == b0,
            prox['lfsr0'] == (curr['lfsr0'] << 1) ^ (s0 & curr['lfsr0']),
            prox['lfsr0'] == curr['lfsr0'])

    t1 = If(majority == b1,
            prox['lfsr1'] == (curr['lfsr1'] << 1) ^ (s1 & curr['lfsr1']),
            prox['lfsr1'] == curr['lfsr1'])

    t2 = If(majority == b2,
            prox['lfsr2'] == (curr['lfsr2'] << 1) ^ (s2 & curr['lfsr2']),
            prox['lfsr2'] == curr['lfsr2'])

    return And(t0, t1, t2)

def init_condition(state):
    """
    Define a condição inicial do sistema.
    
    Args:
        state (dict): Estado atual
    
    Returns:
        Fórmula Z3 representando a condição inicial
    """
    return And(state['lfsr0'] > 0,
              state['lfsr1'] > 0,
              state['lfsr2'] > 0)

def error_condition(state):
    """
    Define a condição de erro do sistema.
    
    Args:
        state (dict): Estado atual
    
    Returns:
        Fórmula Z3 representando a condição de erro
    """
    return Not(init_condition(state))

def get_predecessor(state_formula):
   """
   Calcula estados que podem levar ao estado atual.
   
   Args:
       state_formula: Fórmula do estado que queremos encontrar predecessores
   
   Returns:
       Fórmula que descreve todos estados predecessores válidos
   """
   # Define estado atual como possível predecessor
   curr_state = {'lfsr0': LFSR0, 'lfsr1': LFSR1, 'lfsr2': LFSR2}
   
   # Cria variáveis para próximo estado (estado que conhecemos)
   next_state = get_state_vars("next")
   
   # Substitui variáveis na fórmula para referenciar próximo estado
   next_formula = substitute_vars(state_formula, curr_state, next_state)
   
   # Retorna fórmula que:
   # 1. Segue regras de transição do A5/1 (trans)
   # 2. Leva ao estado que queremos (next_formula)
   return And(trans(curr_state, next_state), next_formula)

def is_valid(formula):
    """
    Verifica se uma fórmula é válida.
    
    Args:
        formula: Fórmula Z3 a ser verificada
    
    Returns:
        bool: True se a fórmula for válida, False caso contrário
    """
    s = Solver()
    s.add(Not(formula))
    return s.check() == unsat

def prove_safety(max_depth=10):
   # Array para armazenar sequência de fórmulas que aproximam estados alcançáveis
   F = []  
   curr_state = {'lfsr0': LFSR0, 'lfsr1': LFSR1, 'lfsr2': LFSR2}
   F.append(init_condition(curr_state))
   
   for k in range(max_depth):
       print(f"Verificando profundidade {k}...")
       
       # Verifica se estado de erro é alcançável no frame atual
       s = Solver()
       s.add(F[k])
       s.add(error_condition(curr_state))
       
       # Se o solver encontrar solução, estado de erro é alcançável - sistema inseguro
       if s.check() == sat:
           print(f"Erro encontrado na profundidade {k}")
           model = s.model()
           print("Contraexemplo:")
           print(f"LFSR0 = {model[LFSR0]}")
           print(f"LFSR1 = {model[LFSR1]}")
           print(f"LFSR2 = {model[LFSR2]}")
           return False
           
       # Inicia novo frame com condições iniciais
       new_frame = init_condition(curr_state)
       
       # Tenta fortalecer frame bloqueando estados predecessores ruins
       attempts = 0
       max_attempts = 100
       while attempts < max_attempts:
           attempts += 1
           # Obtém estados que poderiam levar a estados de erro
           pred = get_predecessor(error_condition(curr_state))
           
           # Verifica se frame já prova que estes estados não podem ser alcançados
           # Se verdadeiro, frame é forte o suficiente para provar segurança neste nível
           if is_valid(Implies(new_frame, Not(pred))):
               break
               
           # Adiciona restrição para bloquear estes estados predecessores
           new_frame = And(new_frame, Not(pred))
       
       F.append(new_frame)
       
       # Verifica se algum frame implica próximo frame (encontrou invariante)
       # Isso prova que o sistema é seguro pois nenhum estado de erro é alcançável
       for i in range(k+1):
           if is_valid(Implies(F[i], F[i+1])):
               print(f"Invariante encontrado na profundidade {k}, frame {i}")
               return True
           
   # Atingiu limite de profundidade sem provar segurança ou encontrar erro
   print("Limite de profundidade alcançado sem conclusão")
   return None

def main():
    """Função principal do programa"""
    print("Iniciando verificação de segurança do A5/1...")
    curr_state = {'lfsr0': LFSR0, 'lfsr1': LFSR1, 'lfsr2': LFSR2}
    print("Condição inicial:", init_condition(curr_state))
    print("Condição de erro:", error_condition(curr_state))
    
    result = prove_safety()
    
    if result is True:
        print("Sistema é seguro - propriedade provada!")
    elif result is False:
        print("Sistema não é seguro - contraexemplo encontrado!")
    else:
        print("Não foi possível determinar segurança com o limite de profundidade dado")

if __name__ == "__main__":
    main()

Iniciando verificação de segurança do A5/1...
Condição inicial: And(lfsr0 > 0, lfsr1 > 0, lfsr2 > 0)
Condição de erro: Not(And(lfsr0 > 0, lfsr1 > 0, lfsr2 > 0))
Verificando profundidade 0...
Verificando profundidade 1...
Invariante encontrado na profundidade 1, frame 1
Sistema é seguro - propriedade provada!
